# Fleet Decision Platform - End-to-End Overview

This notebook is a single, readable walkthrough of the Fleet Decision Platform.
It explains the core concepts and provides a minimal, end-to-end sequence you can follow.

## What you will learn
- How the platform is structured
- How to configure and run the pipeline
- How to use the API and dashboard
- How to run tests and documentation


## Table of contents

1. Architecture and concepts
2. Environment setup
3. Configuration
4. Data acquisition and simulation
5. Optimization quick run
6. API and dashboard
7. Tests and documentation
8. Next steps


## Architecture and concepts

The platform is a modular pipeline:
- **Forecasting** produces demand by zone and time.
- **Optimization** allocates vehicles with a min-cost flow solver.
- **Risk** scores vehicle health and prioritizes maintenance.
- **API** exposes these capabilities for UI and integrations.

Key folders:
- `src/`: core logic and APIs
- `scripts/`: runnable utilities
- `config/`: central configuration
- `docs/`: documentation


## Environment setup

Use `uv` for Python dependency management.


In [1]:
# Install dependencies
!uv sync
!uv sync --all-extras

Resolved 302 packages in 14ms
Uninstalled 60 packages in 3.59s
 - absl-py==2.4.0
 - altair==6.0.0
 - annotated-doc==0.0.4
 - annotated-types==0.7.0
 - attrs==25.4.0
 - babel==2.17.0
 - backrefs==6.1
 - blinker==1.9.0
 - cachetools==6.2.6
 - charset-normalizer==3.4.4
 - click==8.3.1
 - colorama==0.4.6
 - fastapi==0.128.0
 - ghp-import==2.1.0
 - gitdb==4.0.12
 - gitpython==3.1.46
 - httptools==0.7.1
 - immutabledict==4.2.2
 - jinja2==3.1.6
 - joblib==1.5.3
 - jsonschema==4.26.0
 - jsonschema-specifications==2025.9.1
 - markdown==3.10.1
 - markupsafe==3.0.3
 - mergedeep==1.3.4
 - mkdocs==1.6.1
 - mkdocs-get-deps==0.2.0
 - mkdocs-material==9.7.1
 - mkdocs-material-extensions==1.3.1
 - narwhals==2.15.0
 - ortools==9.15.6755
 - paginate==0.5.7
 - pathspec==1.0.4
 - pillow==12.1.0
 - plotly==6.5.2
 - protobuf==6.33.5
 - pydantic==2.12.5
 - pydantic-core==2.41.5
 - pydantic-settings==2.12.0
 - pydeck==0.9.1
 - pymdown-extensions==10.20.1
 - pyyaml-env-tag==1.1
 - referencing==0.37.0
 - request

## Configuration

All runtime defaults come from `config/config.yaml`.
You can inspect it programmatically:


In [2]:
from src.utils.config import get_config


config = get_config()
config["forecasting"].keys()

dict_keys(['model', 'horizon_days', 'horizon_hours', 'features', 'xgboost', 'training'])

## Data acquisition and simulation

Use the Kaggle downloader for real data, or generate synthetic fleet data.


In [3]:
# Download datasets (requires Kaggle credentials)
!uv run python scripts/download_data.py

# Generate synthetic fleet state and network costs
!uv run python scripts/generate_fleet.py --num-vehicles 50 --num-locations 5

2026-02-01 18:42:14,406 - INFO - Kaggle credentials found at ~/.kaggle/kaggle.json
2026-02-01 18:42:14,406 - INFO - Downloading NYC Yellow Taxi Trip Data...
2026-02-01 18:42:14,406 - INFO - Kaggle dataset: elemento/nyc-yellow-taxi-trip-data
2026-02-01 18:42:14,406 - INFO - Output directory: data/raw/nyc_taxi
^C
Traceback (most recent call last):
  File "/Users/ashraf96/Desktop/Data-Driven-Fleet-Cascade/scripts/download_data.py", line 156, in <module>
    main()
  File "/Users/ashraf96/Desktop/Data-Driven-Fleet-Cascade/scripts/download_data.py", line 145, in main
    if not download_dataset(dataset):
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ashraf96/Desktop/Data-Driven-Fleet-Cascade/scripts/download_data.py", line 98, in download_dataset
    result = subprocess.run(cmd, capture_output=True, text=True)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ashraf96/.local/share/uv/python/cpython-3.12.9-macos-aarch64-none/lib/python3.12/subprocess.py", 

## Optimization quick run (in-notebook)

This runs a minimal optimization using synthetic inputs.


In [ ]:
from src.data.loader import generate_demand_forecast, generate_fleet_state, generate_network_costs
from src.optimization.cascade import FleetOptimizer


n_zones = 9
fleet_df = generate_fleet_state(n_vehicles=20, n_zones=n_zones, seed=42)
demand = generate_demand_forecast(n_zones=n_zones, hour=18, day_of_week=4, seed=42)
costs = generate_network_costs(n_zones=n_zones, seed=42)

optimizer = FleetOptimizer(max_cost_per_vehicle=50.0, min_service_level=0.0)
result = optimizer.optimize(fleet_df=fleet_df, demand=demand, costs=costs)

result.status, result.total_cost, result.kpis

## API and dashboard

Run the API and Streamlit dashboard in separate terminals.


In [ ]:
# Start the API (terminal 1)
!uv run uvicorn src.api.main:app --reload

# Start the dashboard (terminal 2)
!uv run streamlit run app.py

## Tests and documentation

Run tests and serve docs locally.


In [ ]:
# Tests
!uv run pytest

# Documentation
!uv run mkdocs serve

## Next steps

- Train a forecasting model and save it under `data/models/`.
- Integrate real data pipelines once Kaggle data is available.
- Extend optimization constraints using `config/config.yaml`.
